In [5]:
!mkdir -p Rawdata
!google-drive-ocamlfuse Rawdata

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [6]:
!wget https://raw.githubusercontent.com/livingbio/DeepLearningTutorial/master/raw_sentences.txt -P 'Rawdata/gliacloud text'

--2018-04-26 17:29:37--  https://raw.githubusercontent.com/livingbio/DeepLearningTutorial/master/raw_sentences.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2955731 (2.8M) [text/plain]
Saving to: ‘Rawdata/gliacloud text/raw_sentences.txt’

raw_sentences.txt   100%[===================>]   2.82M   174KB/s    in 20s     

2018-04-26 17:30:00 (143 KB/s) - ‘Rawdata/gliacloud text/raw_sentences.txt’ saved [2955731/2955731]



In [21]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import numpy as np

def ngram_probs(filename='raw_sentences.txt'):
    with open(filename,'r') as file:
        content_text = file.read()
    
    tokenizer = Tokenizer(lower=True, split=" ")
    tokenizer.fit_on_texts([content_text])
    vocab = tokenizer.word_index
    vocab_size = len(vocab)
    
    vocab_reverse = {}
    vocab_keys = list(vocab.keys())
    for vocab_word in vocab_keys:
        vocab_index = vocab[vocab_word]
        vocab_reverse[vocab_index] = vocab_keys[vocab_index-1]
            
    content_word_ids = tokenizer.texts_to_sequences([content_text])
    content_word_ids = content_word_ids[0]
    content_len = len(content_word_ids)
    
    unigram_probs = np.zeros([vocab_size])
    bigram_probs = np.zeros([vocab_size, vocab_size])
    trigram_probs = np.zeros([vocab_size, vocab_size, vocab_size])
    
    for i in range(content_len):
        # P(W_i)
        unigram_probs[content_word_ids[i]-1] += 1
        
        # P(W_i|W_i-1)
        if i>0:
            bigram_probs[content_word_ids[i]-1, content_word_ids[i-1]-1] += 1
        
        # P(W_i|W_i-2, W_i-1)
        if i-1>0:
            trigram_probs[content_word_ids[i]-1, content_word_ids[i-1]-1, content_word_ids[i-2]-1] += 1
    
    return unigram_probs, bigram_probs, trigram_probs, vocab, vocab_reverse

cnt1, cnt2, cnt3, vocab ,vocab_reverse = ngram_probs('Rawdata/gliacloud text/raw_sentences.txt')
print(len(vocab))
print(cnt2[(vocab['we']-1, vocab['are']-1)])

241
283.0


In [37]:
def prob3(bigram, target, cnt2=cnt2, cnt3=cnt3):
    word_id1 = vocab[bigram[1]]-1
    word_id0 = vocab[bigram[0]]-1
    target_id = vocab[target]-1
    prob = cnt3[:,word_id1, word_id0] / (cnt2[word_id1, word_id0] + 1e-12)
    return np.log(prob[target_id] + 1e-12)

p = prob3(['we', 'are'], 'family')
print(p)

-5.418521710667886


In [38]:
def predict_max(starting, cnt2=cnt2, cnt3=cnt3):
    list_of_words = starting
    while list_of_words[-1]!='.' and len(list_of_words)<15:
        p_word = []
        for word in vocab:
            p_word.append(prob3([list_of_words[-1], list_of_words[-2]], word))
        p_max = np.array(p_word).argmax()
        list_of_words.append(vocab_reverse[p_max+1])
    
    return list_of_words

sent = predict_max(['we', 'are'])
assert sent[-1] == '.' or len(sent) <= 15
print(' '.join(sent))

we are going nt to it be 's it not 's the it way is
